Install the correct dependencies on HuggingFace transformer and ternsorflow

In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
!pip install nlp==0.2.0
!pip install datasets
!pip install git+https://github.com/huggingface/nlp

# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-plp9s8wa
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-plp9s8wa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 6.6 MB 41.4 MB/s 
     |████████████████████████████████| 596 kB 52.4 MB/s 
  Created wheel for transformers: filename=transformers-4.20.0.dev0-py3-none-any.whl size=4171103 sha256=8b39ec060327b98dc94c8a22f4026398eecafcdceea6576419f831cee6db09b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-0z34eq2h/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: P

Train a customised python byte-level Byte-pair encoding tokenizer. 

In [ ]:
from pathlib import Path
from transformers import AutoTokenizer,TextDataset,DataCollatorForLanguageModeling
import glob
import random 

tokenizer = AutoTokenizer.from_pretrained("gpt2")


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": 

In [2]:
def compute_metric():

  nlp.load_metric('accuracy', name="token").compute(
      np.argmax(preds_dict["token"].predictions, axis=1),
      preds_dict["token"].label_ids,
  )

  nlp.load_metric('average_precision_score', name="token").compute(
      np.argmax(preds_dict["token"].predictions, axis=1),
      preds_dict["token"].label_ids,
  )

  nlp.load_metric('bleu', name="line").compute(
      np.argmax(preds_dict["line"].predictions, axis=1),
      preds_dict["line"].label_ids,
  )

  nlp.load_metric('meteor', name="line").compute(
      np.argmax(preds_dict["line"].predictions, axis=1),
      preds_dict["line"].label_ids,
  )

  nlp.load_metric('rouge', name="line").compute(
      np.argmax(preds_dict["line"].predictions, axis=1),
      preds_dict["line"].label_ids,
  )


In [ ]:
paths = [str(x) for x in Path(".").glob("./sample_data/test_data/*.py")]
converted_paths = []
for path in paths:
  converted_path = "./sample_data/test_data/"+ path.split("/").pop().split(".")[0] + ".txt"
  print(converted_path)
  try:
    convert(path, converted_path)
    converted_paths.append(converted_path)
  except:
    pass


with open("./test.txt", "wb") as test_outfile:
  for f in paths:
      with open(f, "rb") as infile:
          test_outfile.write(infile.read())

with open("./converted_test.txt", "wb") as test_outfile:
  for f in converted_paths:
      with open(f, "rb") as infile:
          test_outfile.write(infile.read())



In [ ]:
def load_dataset(test_path,tokenizer):
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return test_dataset,data_collator

test_dataset,data_collator = load_dataset("./train.txt", "./test.txt",tokenizer)
converted_test_dataset, converted_datacollator = load_dataset("./converted_train.txt", "./converted_test.txt",tokenizer)
#pretrain_raw_files = glob.glob("./pretrain_dataset" + '/**/*.py', recursive=True)
#pretrain_converted_files = glob.glob("./pretrain_converted_dataset" + '/**/*.py', recursive=True)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Loading features from cached file ./cached_lm_GPT2TokenizerFast_128_train.txt [took 0.000 s]
Loading features from cached file ./cached_lm_GPT2TokenizerFast_128_test.txt [took 0.000 s]
Loading features from cached file ./cached_lm_GPT2TokenizerFast_128_converted_train.txt [took 0.000 s]
Loading features from cached file ./cached_lm_GPT2TokenizerFast_128_converted_test.txt [took 0.000 s]


In [ ]:
tokenizer("for i in range(10)")["input_ids"]

[1640, 1312, 287, 2837, 7, 940, 8]

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import transformers
import nlp
import logging
from datasets import load_dataset
from transformers import TextDataset,DataCollatorForLanguageModeling


logging.basicConfig(level=logging.INFO)

dataset_dict = {
    "token": train_dataset,
    "token_type": train_dataset,
    "line": train_dataset,
}

print(dataset_dict["token"])


In [ ]:
from transformers.utils.dummy_pt_objects import GPT2LMHeadModel
from transformers import GPT2Tokenizer
from transformers import GPT2Config, EncoderDecoderConfig, EncoderDecoderModel


class MultitaskModel(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        """
        Setting MultitaskModel up as a PretrainedModel allows us
        to take better advantage of Trainer features
        """
        super().__init__(transformers.PretrainedConfig())

        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    def _get_models(self):
      return self.taskmodels_dict

    @classmethod
    def create(cls, model_name, model_type_dict, model_config_dict):
        """
        This creates a MultitaskModel using the model class and config objects
        from single-task models. 

        We do this by creating each single-task model, and having them share
        the same encoder transformer.
        """
        shared_encoder = None
        taskmodels_dict = {}
        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained( "gpt2",
                config=model_config_dict[task_name],
            )
            if shared_encoder is None:
                shared_encoder = cls.get_encoder(model)
            else:
                setattr(model, "encoder", shared_encoder)
            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)
    

    @classmethod
    def get_encoder(cls, model):
        """
        The encoder transformer is named differently in each model "architecture".
        This method lets us get the name of the encoder attribute
        """
        model_class_name = model.__class__.__name__
        if model_class_name.startswith("Roberta"):
            return "roberta-base"
        elif model_class_name.startswith("GPT2"):
            config = EncoderDecoderConfig.from_encoder_decoder_configs(model.config, model.config) 
            encoder_decoder = EncoderDecoderModel(config=config)
            return encoder_decoder.config.encoder
        else:
            raise KeyError(f"Add support for new model {model_class_name}")
    
    def forward(self, task_name, **kwargs):
        return self.taskmodels_dict[task_name](**kwargs)

In [ ]:
model_name = "gpt2"
multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        "token": transformers.AutoModelWithLMHead,
        "token_type": transformers.AutoModelWithLMHead,
        "line": transformers.AutoModelForSequenceClassification,
    },
    model_config_dict={
        "token": transformers.AutoConfig.from_pretrained(model_name),
        "token_type": transformers.AutoConfig.from_pretrained(model_name),
        "line": transformers.AutoConfig.from_pretrained(model_name),
    },
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Check that we have a GPU
!nvidia-smi
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



False

In [ ]:
import dataclasses
from torch.utils.data.dataloader import DataLoader
from transformers.data.data_collator import DataCollatorForLanguageModeling, InputDataClass, DefaultDataCollator
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from typing import List, Union, Dict
from transformers import Trainer
from random import random


class NLPDataCollator(DataCollatorForLanguageModeling):
    """
    Extending the existing DataCollator to work with NLP dataset batches
    """
    def collate_batch(self, features: List[Union[InputDataClass, Dict]]) -> Dict[str, torch.Tensor]:
        first = features[0]
        if isinstance(first, dict):
          # NLP data sets current works presents features as lists of dictionary
          # (one per example), so we  will adapt the collate_batch logic for that
          if "labels" in first and first["labels"] is not None:
              if first["labels"].dtype == torch.int64:
                  labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
              else:
                  labels = torch.tensor([f["labels"] for f in features], dtype=torch.float)
              batch = {"labels": labels}
          for k, v in first.items():
              if k != "labels" and v is not None and not isinstance(v, str):
                  batch[k] = torch.stack([f[k] for f in features])
          return batch
        else:
          # otherwise, revert to using the default collate_batch
          return DefaultDataCollator().collate_batch(features)


class StrIgnoreDevice(str):
    """
    This is a hack. The Trainer is going call .to(device) on every input
    value, but we need to pass in an additional `task_name` string.
    This prevents it from throwing an error
    """
    def to(self, device):
        return self

class DataLoaderWithTaskname:
    """
    Wrapper around a DataLoader to also yield a task name
    """
    def __init__(self, task_name, data_loader):
        self.task_name = task_name
        self.data_loader = data_loader

        self.batch_size = data_loader.batch_size
        self.dataset = data_loader.dataset

    def __len__(self):
        return len(self.data_loader)
    
    def __iter__(self):
        for batch in self.data_loader:
            batch["task_name"] = StrIgnoreDevice(self.task_name)
            yield batch


class MultitaskDataloader:
    """
    Data loader that combines and samples from multiple single-task
    data loaders.
    """
    def __init__(self, dataloader_dict):
        self.dataloader_dict = dataloader_dict
        self.num_batches_dict = {
            task_name: len(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        self.task_name_list = list(self.dataloader_dict)
        self.dataset = [None] * sum(
            len(dataloader.dataset) 
            for dataloader in self.dataloader_dict.values()
        )

    def __len__(self):
        return sum(self.num_batches_dict.values())

    def __iter__(self):
        """
        For each batch, sample a task, and yield a batch from the respective
        task Dataloader.

        We use size-proportional sampling, but you could easily modify this
        to sample from some-other distribution.
        """
        task_choice_list = []
        for i, task_name in enumerate(self.task_name_list):
            task_choice_list += [i] * self.num_batches_dict[task_name]
        task_choice_list = np.array(task_choice_list)
        np.random.shuffle(task_choice_list)
        dataloader_iter_dict = {
            task_name: iter(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        for task_choice in task_choice_list:
            task_name = self.task_name_list[task_choice]
            yield next(dataloader_iter_dict[task_name]) 

class MultitaskTrainer(transformers.Trainer):

    def get_single_train_dataloader(self, task_name, train_dataset):
        """
        Create a single-task data loader that also yields task names
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        
        train_sampler = (
            RandomSampler(train_dataset)
            if self.args.local_rank == -1
            else DistributedSampler(train_dataset)
        )

        data_loader = DataLoaderWithTaskname(
            task_name=task_name,
            data_loader=DataLoader(
              train_dataset,
              batch_size=self.args.train_batch_size,
              sampler=train_sampler
            ),
        )

        return data_loader

    def get_train_dataloader(self):
        """
        Returns a MultitaskDataloader, which is not actually a Dataloader
        but an iterable that returns a generator that samples from each 
        task Dataloader
        """
        return MultitaskDataloader({
            task_name: self.get_single_train_dataloader(task_name, task_dataset)
            for task_name, task_dataset in self.train_dataset.items()
        })
    
    def train(self):
      config = transformers.AutoConfig.from_pretrained("gpt2")
      model = transformers.AutoModelWithLMHead.from_pretrained("gpt2", config=config)
      self.trainer = Trainer(
        model=model,
        args=transformers.TrainingArguments(
          output_dir="./models/multitask_model",
          overwrite_output_dir=True,
          learning_rate=1e-5,
          do_train=True,
          num_train_epochs=100,
          # Adjust batch size if this doesn't fit on the Colab GPU
          per_device_train_batch_size=8,  
          save_steps=3000,
        ),
        data_collator=data_collator,
        train_dataset=train_dataset,
      )
      self.trainer.train()
    
    def evaluate(self):
      config = transformers.AutoConfig.from_pretrained("gpt2")
      model = transformers.AutoModelWithLMHead.from_pretrained("gpt2", config=config)
      self.trainer = Trainer(
        model=model,
        args=transformers.TrainingArguments(
          output_dir="./models/multitask_model",
          overwrite_output_dir=True,
          learning_rate=1e-5,
          do_train=True,
          num_train_epochs=100,
          # Adjust batch size if this doesn't fit on the Colab GPU
          per_device_train_batch_size=8,  
          save_steps=3000,
          compute_metrics=compute_metrics
        ),
        data_collator=data_collator,
        train_dataset=train_dataset,
      )
      self.trainer.train()

    #run evaluation first to set the compute metrics to compute full set of metrics  
    def prediction_loop(self):
      return self.trainer.predict()

    def compute_loss(self, model, inputs, return_outputs=True):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        reranking_layer(outputs, inputs._get_value(), tokenizer=tokenizer) #input value is tensor
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
# point to training folder  
model_path = "./models/multitask_model"
model = AutoModelWithLMHead.from_pretrained(model_path)

# Define test trainer
test_trainer = Trainer(model)
trainer.train()



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upca

ValueError: ignored

In [ ]:
preds_dict = {}
for task_name in ["token", "token_type", "line"]:
      eval_dataloader = DataLoaderWithTaskname(
          task_name,
          trainer.get_eval_dataloader(eval_dataset=dataset_dict[task_name])
      )
      print(eval_dataloader.data_loader.collate_fn)
      preds_dict[task_name] = trainer.prediction_loop(
          eval_dataloader, 
          description=f"Validation: {task_name}",
      )


print(preds_dict)

***** Running Validation: token *****
  Num examples = 0
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/transformers/trainer_pt_utils.py:395: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  FutureWarning,
***** Running Validation: token_type *****
  Num examples = 0
  Batch size = 8
***** Running Validation: line *****
  Num examples = 0
  Batch size = 8


In [ ]:
from sklearn.metrics import accuracy_score, label_ranking_average_precision_score

accuracy_dict = {}
mrr_dict = {}

for task_name in ["token", "token_type", "line"]:
  accuracy_dict[task_name] = accuracy_score(preds_dict[task_name].predictions.flatten(),
    preds_dict[task_name].label_ids)
  
  mrr_dict[task_name] = label_ranking_average_precision_score(preds_dict[task_name].predictions.flatten(),
    preds_dict[task_name].label_ids)
  
  